In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
# This is need so you can import larndsim without doing python setup.py install
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,'/sdf/home/c/cyifan/larnd-sim_checkpoint/larnd-sim')

In [3]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mpl_toolkits.mplot3d.art3d as art3d

import numpy as np
import eagerpy as ep
import h5py

import matplotlib as mpl
#mpl.rcParams['figure.dpi'] = 100
import pickle

In [4]:
from numpy.lib import recfunctions as rfn
import torch

def torch_from_structured(tracks):
    tracks_np = rfn.structured_to_unstructured(tracks, copy=True, dtype=np.float32)
    return torch.from_numpy(tracks_np).float()

def structered_from_torch(tracks_torch, dtype):
    return rfn.unstructured_to_structured(tracks_torch.cpu().numpy(), dtype=dtype)

### Dataset import
First of all we load the `edep-sim` output. For this sample we need to invert $z$ and $y$ axes.

In [5]:
# If you have access to a GPU, sim works trivially/is much faster
if torch.cuda.is_available():
    device = 'cuda'
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = 'cpu'

In [6]:
dir_name = '/sdf/group/neutrino/cyifan/muon-sim/fake_data_S1/edepsim-output.h5'
fname = dir_name
with h5py.File(fname, 'r') as f:
    tracks = np.array(f['segments'])  

x_start = np.copy(tracks['x_start'] )
x_end = np.copy(tracks['x_end'])
x = np.copy(tracks['x'])

tracks['x_start'] = np.copy(tracks['z_start'])
tracks['x_end'] = np.copy(tracks['z_end'])
tracks['x'] = np.copy(tracks['z'])

tracks['z_start'] = x_start
tracks['z_end'] = x_end
tracks['z'] = x

In [7]:
# Let's say we only take tracks that are longer than roughly 5 cm 
# (we shall see if this causes problem for S1)

index = {}
all_events = np.unique(tracks['eventID'])
total_n_trks = 0
for ev in all_events:
    trk_pool = []
    track_set = np.unique(tracks[tracks['eventID'] == ev]['trackID'])
    for trk in track_set:
        trk_msk = (tracks['eventID'] == ev) & (tracks['trackID'] == trk)
        if sum(tracks[trk_msk]['dx']) > 5:
            trk_pool.append(trk)
    index[ev] = np.array(trk_pool)
    total_n_trks += len(trk_pool)
    
print("number of qualified tracks in this sample: " + str(total_n_trks))

# Copy it, so the lottery box can be executed multiple times by itself
temp_index = index

number of qualified tracks in this sample: 627


In [8]:
# Define number of tracks to train with
n_tracks_to_train = 2

In [9]:
# Time to draw lotteries
train_index = []
for i_trk in range(n_tracks_to_train):
    rand_ev = np.random.choice(list(temp_index.keys()))
    if temp_index[rand_ev].size == 0:
        temp_index.pop(rand_ev, None)
        rand_ev = np.random.choice(list(temp_index.keys()))
    rand_track = np.random.choice(temp_index[rand_ev])
    
    temp_index[rand_ev] = np.delete(temp_index[rand_ev],np.where(temp_index[rand_ev] == rand_track)) 
    train_index.append(np.array([rand_ev, rand_track])) 

train_index = np.array(train_index)


In [35]:
def batch(tracks, size=5):
    global i_batch_in_track
    global active_train_index
  
    print("ev: " + str(active_train_index[0][0]) + ", trk: " + str(active_train_index[0][1]) + ", i_batch_in_track: " + str(i_batch_in_track))
    mask = (tracks['eventID']== active_train_index[0][0]) & (tracks['trackID'] == active_train_index[0][1])
    
    if tracks[mask].size > i_batch_in_track * size:
        this_out_trk = tracks[mask][i_batch_in_track * size: (i_batch_in_track + 1) * size]
        out_trk = torch_from_structured(this_out_trk.copy())
        
        i_batch_in_track += 1
        if tracks[mask].size < (i_batch_in_track + 1) * size:
            i_batch_in_track = 0
            active_train_index = np.delete(active_train_index, 0, 0)

    return out_trk 

## Simulation
To flexibly keep track of parameters/gradients, simulations are housed in a class `sim_with_grad`. This is derived from class versions of all the other modules. Parameters are housed in `consts`, with method `track_gradients` to promote the constants to `requires_grad=True` PyTorch tensors.

In [11]:
from larndsim.sim_with_grad import sim_with_grad

## The simulation
Following the flow of the simulation chain, define a function which takes in the `sim_with_grad` object, runs whatever pieces of the simulation, and returns desired output.

In [34]:
def get_id_map(selected_tracks, fields):
#     print(fields)
    # Here we build a map between tracks and event IDs (no param dependence, so np should be ok)
    unique_eventIDs = np.unique(selected_tracks[:, fields.index('eventID')])
    event_id_map = np.searchsorted(unique_eventIDs,np.asarray(selected_tracks[:, fields.index('eventID')]))
    
    event_id_map_torch = torch.from_numpy(event_id_map).to(device)
#     print(unique_eventIDs)
    return event_id_map_torch, unique_eventIDs

In [13]:
def all_sim(sim, selected_tracks, fields, event_id_map, unique_eventIDs, return_unique_pix=False):
    selected_tracks_quench = sim.quench(selected_tracks, sim.birks, fields=fields)
    selected_tracks_drift = sim.drift(selected_tracks_quench, fields=fields)

    active_pixels_torch, neighboring_pixels_torch, n_pixels_list_ep = sim.get_pixels(selected_tracks_drift,
                                                                                     fields=fields)

    track_starts_torch, max_length_torch = sim.time_intervals(event_id_map, 
                                                              selected_tracks_drift, 
                                                              fields=fields)
    
    signals_ep = sim.tracks_current(neighboring_pixels_torch, selected_tracks_drift, 
                                          max_length_torch,
                                          fields=fields)

    unique_pix_torch = torch.empty((0, 2))
    pixels_signals_torch = torch.zeros((len(unique_pix_torch), len(sim.time_ticks)*50))

    shapes_torch = neighboring_pixels_torch.shape
    joined_torch = neighboring_pixels_torch.reshape(shapes_torch[0]*shapes_torch[1], 2)

    this_unique_pix_torch = torch.unique(joined_torch, dim=0)
    this_unique_pix_torch = this_unique_pix_torch[(this_unique_pix_torch[:,0] != -1) & (this_unique_pix_torch[:,1] != -1),:]
    unique_pix_torch = torch.cat((unique_pix_torch, this_unique_pix_torch),dim=0)

    this_pixels_signals_torch = torch.zeros((len(this_unique_pix_torch), len(sim.time_ticks)*50))
    pixels_signals_torch = torch.cat((pixels_signals_torch, this_pixels_signals_torch), dim=0)

    pixel_index_map_torch = torch.full((selected_tracks.shape[0], neighboring_pixels_torch.shape[1]), -1)
    compare_torch = (neighboring_pixels_torch[..., np.newaxis, :] == unique_pix_torch)

    indices_torch = torch.where(torch.logical_and(compare_torch[..., 0], compare_torch[...,1]))
    pixel_index_map_torch[indices_torch[0], indices_torch[1]] = indices_torch[2]
    
    pixels_signals_torch = sim.sum_pixel_signals(pixels_signals_torch,
                                                 signals_ep,
                                                track_starts_torch,
                                                pixel_index_map_torch)
    
    time_ticks_torch = torch.linspace(0, len(unique_eventIDs)*sim.time_interval[1]*3, pixels_signals_torch.shape[1]+1)

    integral_list_torch, adc_ticks_list_torch = sim.get_adc_values(pixels_signals_torch,
                                                                   time_ticks_torch,
                                                                   0)
    adc_list_torch = sim.digitize(integral_list_torch)

    if return_unique_pix:
        return adc_list_torch, unique_pix_torch,
    else:
        return adc_list_torch

In [14]:
# Update parameters for training loop
def update_grad_param(sim, name, value):
    setattr(sim, name, value)
    sim.track_gradients([name])

In [15]:
# ADC counts given as list of pixels. Better for loss to embed this in the "full" pixel space
def embed_adc_list(sim, adc_list, unique_pix):
    zero_val = sim.digitize(torch.tensor(0)).item()
    new_list = torch.ones((sim.n_pixels[0], sim.n_pixels[1], adc_list.shape[1]))*zero_val

    plane_id = unique_pix[..., 0] // sim.n_pixels[0]
    unique_pix[..., 0] = unique_pix[..., 0] - sim.n_pixels[0] * plane_id

    new_list[unique_pix[:, 0].long(), unique_pix[:, 1].long(), :] = adc_list
    
    return new_list

## Define dict with ranges from the spreadsheet
https://docs.google.com/spreadsheets/d/1DLpSDgPsHeHUWCEBayYCcbLzIzd30vfBe72N-Z5vWTc/edit#gid=1247026028

In [17]:
ranges = {}
ranges['lArDensity']     = {'nom': 1.38, 'down': 1.37, 'up': 1.41}
ranges['eField']         = {'nom': 0.5, 'down': 0.45, 'up': 0.55}
ranges['vdrift']         = {'nom': 0.1648, 'down': 0.1400, 'up': 0.1800}
ranges['MeVToElectrons'] = {'nom': 4.24e4, 'down': 3.48e4, 'up': 5.13e4}
ranges['alpha']          = {'nom': 0.93, 'down': 0.85, 'up': 1.1}
ranges['beta']           = {'nom': 0.207, 'down': 0.18, 'up': 0.22}
ranges['Ab']             = {'nom': 0.8, 'down': 0.78, 'up': 0.88}
ranges['kb']             = {'nom': 0.0486, 'down': 0.04, 'up': 0.07}
ranges['lifetime']       = {'nom': 2.2e3, 'down': 300, 'up': 3e4}
ranges['long_diff']      = {'nom': 4.0e-6, 'down': 2e-6, 'up': 9e-6}
ranges['tran_diff']      = {'nom': 8.8e-6, 'down': 4e-6, 'up': 14e-6}

In [75]:
def param_l2_reg(param, sim):
    sigma = (ranges[param]['up'] - ranges[param]['down'])/2.
    return ((ranges[param]['nom']-getattr(sim, param))**2)/(sigma**2)

In [18]:
def calc_reg_loss(param_list, sim, regs):
    reg_loss = 0.
    for param in param_list:
        reg_loss+=regs[param]*param_l2_reg(param, sim)
        
    return reg_loss

In [19]:
#Simulate with some set:
sim_target = sim_with_grad(track_chunk=1, pixel_chunk=1)
sim_target.load_detector_properties("../larndsim/detector_properties/module0.yaml",
                             "../larndsim/pixel_layouts/multi_tile_layout-2.2.16.yaml")

In [20]:
relevant_params = ['eField', 'lifetime']

In [21]:
#Setup simulation object for training -- params initialized to defaults
sim = sim_with_grad(track_chunk=1, pixel_chunk=1)
sim.load_detector_properties("../larndsim/detector_properties/module0.yaml",
                             "../larndsim/pixel_layouts/multi_tile_layout-2.2.16.yaml")

sim.track_gradients(relevant_params)

## Instead of varying all params by hand, draw randomly in range
These are used as the targets

In [22]:
np.random.seed(2)
for param in relevant_params:
    param_val = np.random.uniform(low=ranges[param]['down'], 
                                      high=ranges[param]['up'])
    
    setattr(sim_target, param, param_val)

In [23]:
for param in relevant_params:
    print(f'{param}, target: {getattr(sim_target, param)}, init {getattr(sim, param).item()}')

eField, target: 0.4935994902142004, init 0.5
lifetime, target: 1070.0090852883727, init 2200.0


### Add in rough checkpointing

In [24]:
do_checkpoint=False

In [25]:
from glob import glob
import os

In [26]:
if do_checkpoint:
    saved = glob('history_epoch*.pkl')
    num = max([int(os.path.splitext(file)[0][len('history_epoch'):]) for file in saved])
    history = pickle.load(open(f'history_epoch{num}.pkl', "rb"))

In [27]:
#Setup simulation object for training -- params initialized to defaults
sim = sim_with_grad(track_chunk=1, pixel_chunk=1)
sim.load_detector_properties("../larndsim/detector_properties/module0.yaml",
                             "../larndsim/pixel_layouts/multi_tile_layout-2.2.16.yaml")
if do_checkpoint:
    for param in relevant_params:
        setattr(sim, param, history[param][-1])

In [28]:
#Setup simulation object for training -- params initialized to defaults
sim_extra = sim_with_grad(track_chunk=1, pixel_chunk=1)
sim_extra.load_detector_properties("../larndsim/detector_properties/module0.yaml",
                             "../larndsim/pixel_layouts/multi_tile_layout-2.2.16.yaml")

In [29]:
regs = {}
regs['eField'] = 1e-4
regs['lifetime'] = 1e-4
regs['vdrift'] = 1e-4
regs['lArDensity'] = 1e-4
regs['Ab'] = 1e-4
regs['kb'] = 1e-4

In [30]:
if not do_checkpoint:
    for param in relevant_params:
        setattr(sim, param, getattr(sim, param)/ranges[param]['nom'])
    
sim.track_gradients(relevant_params)

In [31]:
#Simple MSE loss between target and output
loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.SGD([#{'params' : sim.lArDensity, 'lr': 0.2},
                              {'params' : sim.eField, 'lr': 8e0},
                              #{'params' : sim.vdrift, 'lr': 8e0},
                             # {'params' : sim.MeVToElectrons, 'lr': 0.001},
#                               {'params' : sim.Ab, 'lr': 8e0},
#                               {'params' : sim.kb, 'lr': 8e0},
                              {'params' : sim.lifetime, 'lr': 8e0},
#                              #{'params' : sim.long_diff, 'lr': 1e-9},
                              #{'params' : sim.tran_diff, 'lr': 5e-9}
                              ])

                              

training_step_track = {}
for param in relevant_params:
    training_step_track[param] = []
losses = []
reg_losses = []

for param in relevant_params:
    if do_checkpoint:
        training_step_track[param] += history[param]
    else:
        training_step_track[param].append(getattr(sim, param).item())

In [32]:
from tqdm import tqdm

In [37]:
# n_batch = 10

#Training loop
track_mem = []
save_freq = 5
err_count = 0

i_batch_in_track = 0

for epoch in range(4):  
    print("--- epoch: ", epoch)
    
    active_train_index = np.copy(train_index)
    while(active_train_index.size > 0):
        losses_batch=[]
        optimizer.zero_grad()
        selected_tracks_torch = batch(tracks, size = 5)
      
        event_id_map, unique_eventIDs = get_id_map(selected_tracks_torch, tracks.dtype.names)
        selected_tracks_torch = selected_tracks_torch.to(device)
        
        target, pix_target = all_sim(sim_target, selected_tracks_torch, tracks.dtype.names, 
                                     event_id_map, unique_eventIDs,
                                  return_unique_pix=True)
        
        for param in relevant_params:
            setattr(sim_extra, param, getattr(sim, param)*ranges[param]['nom'])

        #Simulate with that parameter and get output
        output, pix_out = all_sim(sim_extra, selected_tracks_torch, tracks.dtype.names, 
                                  event_id_map, unique_eventIDs,
                                  return_unique_pix=True)
        
        embed_output = embed_adc_list(sim, output, pix_out)
        embed_target = embed_adc_list(sim_target, target, pix_target)
        mem = torch.cuda.memory_allocated()/(1024*1024)
        track_mem.append(mem)
    
        #Calc loss between simulated and target + backprop
        loss = loss_fn(embed_output, embed_target) #+ calc_reg_loss(relevant_params, sim, regs)
        loss.backward()
    
        nan_check = torch.tensor([getattr(sim, param).grad.isnan() for param in relevant_params]).sum()
        if nan_check == 0 and loss !=0 and not loss.isnan():
            optimizer.step()
            losses_batch.append(loss.item())
                
    if len(losses_batch) > 0:
        losses.append(np.mean(losses_batch))
        print(np.mean(losses_batch))
    if epoch % 1 == 0:
        for param in relevant_params:
            print(param, getattr(sim,param).item())
        
    for param in relevant_params:
        #print(param, getattr(sim_target, param), getattr(sim, param).item())
        training_step_track[param].append(getattr(sim, param).item())
    
    n_steps = len(training_step_track[param])
    if n_steps % save_freq == 0:
        with open(f'log/history_epoch{n_steps}.pkl', "wb") as f_history:
            pickle.dump(training_step_track, f_history)
            pickle.dump(losses, f_history)

--- epoch:  0
ev: 8, trk: 8, i_batch_in_track: 0
ev: 8, trk: 8, i_batch_in_track: 1
ev: 8, trk: 8, i_batch_in_track: 2
ev: 8, trk: 8, i_batch_in_track: 3
ev: 1, trk: 5, i_batch_in_track: 0
ev: 1, trk: 5, i_batch_in_track: 1
ev: 1, trk: 5, i_batch_in_track: 2
0.0006475239060819149
eField 0.9261345267295837
lifetime 0.9617841243743896
--- epoch:  1
ev: 8, trk: 8, i_batch_in_track: 0
ev: 8, trk: 8, i_batch_in_track: 1
ev: 8, trk: 8, i_batch_in_track: 2
ev: 8, trk: 8, i_batch_in_track: 3
ev: 1, trk: 5, i_batch_in_track: 0
ev: 1, trk: 5, i_batch_in_track: 1
ev: 1, trk: 5, i_batch_in_track: 2
0.000716955226380378
eField 0.8940661549568176
lifetime 0.9454888105392456
--- epoch:  2
ev: 8, trk: 8, i_batch_in_track: 0
ev: 8, trk: 8, i_batch_in_track: 1
ev: 8, trk: 8, i_batch_in_track: 2
ev: 8, trk: 8, i_batch_in_track: 3
ev: 1, trk: 5, i_batch_in_track: 0
ev: 1, trk: 5, i_batch_in_track: 1
ev: 1, trk: 5, i_batch_in_track: 2
0.0009431554353795946
eField 0.8640632629394531
lifetime 0.9308220148086

In [ ]:
for param in relevant_params:
    plt.plot(np.asarray(training_step_track[param])*ranges[param]['nom']/getattr(sim_target, param))
    plt.plot([0, len(training_step_track[param])], [1, 1], c='k', ls='dashed')
    plt.ylabel('Fitted Value / Target')
    plt.title(param)
    plt.xlabel('Training Epoch')
    plt.savefig(f'snapshot_{param}_with_normalize_S0_1000.pdf')
    plt.show()

In [ ]:
plt.plot(np.asarray(losses))
plt.ylabel('Loss')
plt.title(param)
plt.xlabel('Training Epoch')
plt.savefig(f'snapshot_loss_with_normalize_S0_1000.pdf')
plt.show()

In [ ]:
plt.scatter(np.asarray(training_step_track['eField'])*ranges['eField']['nom'],
            np.asarray(training_step_track['lifetime'])*ranges['lifetime']['nom']
           )
plt.scatter(getattr(sim_target, 'eField'), getattr(sim_target, 'lifetime'), marker=(5, 1), label='target')
plt.scatter(np.asarray(training_step_track['eField'][0])*ranges['eField']['nom'],
            np.asarray(training_step_track['lifetime'][0])*ranges['lifetime']['nom'], 
            marker="<", label='start')

legend = plt.legend(loc='upper right')
plt.savefig(f'snapshot_eField_lifetime_with_normalize_S0_1000.pdf')
plt.show()